# Проект Маркетинг

## Описание

Интернет-магазин собирает историю покупателей, проводит рассылки предложений и планирует будущие продажи. Для оптимизации процессов надо выделить пользователей, которые готовы совершить покупку в ближайшее время.

### Цель

Предсказать вероятность покупки в течение 90 дней

### Задачи

● Изучить данные  
● Разработать полезные признаки  
● Создать модель для классификации пользователей  
● Улучшить модель и максимизировать метрику roc_auc  
● Выполнить тестирование  

### Данные

apparel-purchases  
история покупок  
● client_id идентификатор пользователя  
● quantity количество товаров в заказе  
● price цена товара  
● category_ids вложенные категории, к которым отнсится товар  
● date дата покупки  
● message_id идентификатор сообщения из рассылки  

apparel-messages  
история рекламных рассылок  
● bulk_campaign_id идентификатор рекламной кампании  
● client_id идентификатор пользователя  
● message_id идентификатор сообщений  
● event тип действия  
● channel канал рассылки  
● date дата рассылки  
● created_at точное время создания сообщения  

apparel-target_binary  
совершит ли клиент покупку в течение следующих 90 дней  
● client_id идентификатор пользователя  
● target целевой признак  


## Подготовка к работе

### Импорты

In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
from IPython.display import HTML, display
import seaborn as sns
from datetime import datetime
from tqdm import tqdm
import time
import re
import ast

### Константы

In [137]:
RANDOM_STATE = 20

### Функции

In [138]:
# форматирования текста
def format_display(text):
    return HTML(f"<span style='font-size: 1.5em; font-weight: bold; font-style: italic;'>{text}</span>")

# сделаем функцию оценки пропусков в датасетах
def missing_data(data):
    missing_data = data.isna().sum()
    missing_data = missing_data[missing_data > 0]
    display(missing_data)

# функция для обработки пробелов
def process_spaces(s):
    if isinstance(s, str):
        s = s.strip()
        s = ' '.join(s.split())
    return s

# замена пробелов на нижнее подчеркинвание в названии столбцов
def replace_spaces(s):
    if isinstance(s, str):
        s = s.strip()
        s = '_'.join(s.split())
    return s

def drop_duplicated(data):
    # проверка дубликатов
    display(format_display("Проверим дубликаты и удалим, если есть"))
    num_duplicates = data.duplicated().sum()
    display(num_duplicates)
    
    if num_duplicates > 0:
        display("Удаляем")
        data = data.drop_duplicates(keep='first').reset_index(drop=True)  # обновляем DataFrame
    else:
        display("Дубликаты отсутствуют")
    return data

def normalize_columns(columns):
    new_cols = []
    for col in columns:
        # вставляем "_" перед заглавной буквой (латиница или кириллица), кроме первой
        col = re.sub(r'(?<!^)(?=[A-ZА-ЯЁ])', '_', col)
        # приводим к нижнему регистру
        col = col.lower()
        new_cols.append(col)
    return new_cols

def check_data(data):
    # приведем все к нижнему регистру
    data.columns = normalize_columns(data.columns)
    
    # удалим лишние пробелы в строках
    data = data.map(process_spaces)

    # и в названии столбцов
    data.columns = [replace_spaces(col) for col in data.columns]
    
    # общая информация 
    display(format_display("Общая информация базы данных"))
    display(data.info())
    
    # 5 строк
    display(format_display("5 случайных строк"))
    display(data.sample(5))
    
    # пропуски
    display(format_display("Число пропусков в базе данных"))
    display(missing_data(data))

    # проверка на наличие пропусков
    if data.isnull().sum().sum() > 0:
        display(format_display("Визуализация пропусков"))
        msno.bar(data)
        plt.show()
        
    # средние характеристики
    display(format_display("Характеристики базы данных"))
    display(data.describe().T)
    
    # data = drop_duplicated(data)
    
    return data  # возвращаем измененные данные

def parse_category_ids(x):
    if isinstance(x, str):
        return ast.literal_eval(x)
    return x

# def safe_mode(x: pd.Series):
#     """Для разбора категорий товара, чтобы не падал код и возвращало NOne если пусто"""
#     m = x.mode()
#     return m.iloc[0] if not m.empty else None

# def last_non_empty_cat(row, max_depth):
#     """Берёт последнее непустое значение из cat_lvl1…cat_lvlN"""
#     for i in range(max_depth, 0, -1):  # от N к 1
#         val = row.get(f"cat_lvl{i}")
#         if pd.notna(val):
#             return val
#     return None

## EDA

### Подключим и почитаем данные

In [139]:
app_msg = pd.read_csv("../data/apparel-messages.csv")
app_prch = pd.read_csv("../data/apparel-purchases.csv")
app_target = pd.read_csv("../data/apparel-target_binary.csv")
event_type = pd.read_csv("../data/full_campaign_daily_event.csv")
event_chanel = pd.read_csv("../data/full_campaign_daily_event_channel.csv")

### Первичная оценка и обработка данных

#### app_msg

In [140]:
app_msg = check_data(app_msg)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12739798 entries, 0 to 12739797
Data columns (total 7 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   bulk_campaign_id  int64 
 1   client_id         int64 
 2   message_id        object
 3   event             object
 4   channel           object
 5   date              object
 6   created_at        object
dtypes: int64(2), object(5)
memory usage: 680.4+ MB


None

,bulk_campaign_id,client_id,message_id,event,channel,date,created_at
2560009,7845,1515915625468102400,1515915625468102400-7845-6373359a03ed7,purchase,email,2022-11-15,2022-11-15 15:52:10
4165576,10438,1515915625488010004,1515915625488010004-10438-63f7474467acb,open,mobile_push,2023-02-23,2023-02-23 11:03:18
2916183,8434,1515915625490740149,1515915625490740149-8434-638f2ed59b295,send,mobile_push,2022-12-06,2022-12-06 12:12:49
658340,5335,1515915625479850592,1515915625479850592-5335-62c7d2ab6b7fb,send,email,2022-07-08,2022-07-08 06:48:23
188696,4679,1515915625539302643,1515915625539302643-4679-6297223ee490f,open,email,2022-06-01,2022-06-01 15:09:08


Series([], dtype: int64)

None

,count,mean,std,min,25%,50%,75%,max
bulk_campaign_id,12739798.0,1.160459e+04,3.259211e+03,5.480000e+02,8.746000e+03,1.351600e+04,1.415800e+04,1.465700e+04
client_id,12739798.0,1.515916e+18,1.329704e+08,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18


In [141]:
# посмотрим дату начала и конца событий
display(app_msg['date'].min())
display(app_msg['date'].max())

'2022-05-19'

'2024-02-15'

In [142]:
# посмотрим уники среди событий и канал распространения
display(app_msg['event'].unique())
display(app_msg['channel'].unique())

array(['open', 'click', 'purchase', 'send', 'unsubscribe', 'hbq_spam',
       'hard_bounce', 'subscribe', 'soft_bounce', 'complain', 'close'],
      dtype=object)

array(['email', 'mobile_push'], dtype=object)

Что имеем:  
open — письмо открыто  
click — клик по ссылке в письме  
purchase — покупка после перехода из письма  
send — отправка письма  
unsubscribe — отписка от рассылки  
hbq_spam — сообщение отмечено как спам  
hard_bounce — письмо не доставлено из-за постоянной ошибки (адрес не существует)  
subscribe — подписка на рассылку  
soft_bounce — письмо не доставлено из-за временной ошибки (ящик переполнен, сервер недоступен)  
complain — жалоба пользователя (напр. “Это спам”)  
close — завершение сессии (иногда: закрытие письма или вкладки)  

In [143]:
# проверим сколько уникальных клиентов
display(app_msg['client_id'].nunique())

53329

Из 12.739.798 строк мы имеем 53.329 уникальных клиентов.  
Нужна будет пересборка данных с агрегацией

Выводы:  
Самая объемная база.  
Столбцы  date, created_at имеют неверный формат данных - необходимо будет преобразовать.


#### app_prch

In [144]:
app_prch = check_data(app_prch)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202208 entries, 0 to 202207
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   client_id     202208 non-null  int64  
 1   quantity      202208 non-null  int64  
 2   price         202208 non-null  float64
 3   category_ids  202208 non-null  object 
 4   date          202208 non-null  object 
 5   message_id    202208 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 9.3+ MB


None

,client_id,quantity,price,category_ids,date,message_id
200632,1515915625821124009,1,12.0,[],2024-02-12,1515915625821124009-14623-65c5c5ad57c63
197466,1515915625768963587,1,1299.0,"['4', '27', '142', '496']",2024-01-26,1515915625768963587-14535-65b210e250849
13829,1515915625475134951,1,699.0,"['4', '28', '124', '415']",2022-06-16,1515915625475134951-4918-62a97ced99478
198472,1515915626005691446,1,399.0,"['4', '28', '260', '420']",2024-01-31,1515915626005691446-14561-65b7a1ce4b7ff
128039,1515915625602828124,1,1999.0,"['4', '28', '244', '432']",2023-06-03,1515915625809747812-13076-6479c4561adf1


Series([], dtype: int64)

None

,count,mean,std,min,25%,50%,75%,max
client_id,202208.0,1.515916e+18,1.459458e+08,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18
quantity,202208.0,1.006483e+00,1.843837e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+01
price,202208.0,1.193302e+03,1.342253e+03,1.000000e+00,3.520000e+02,9.870000e+02,1.699000e+03,8.549900e+04


In [145]:
# проверим сколько уникальных клиентов совершило покупки
display(app_prch['client_id'].nunique())

49849

In [146]:
display(app_prch['client_id'].nunique() / app_msg['client_id'].nunique())

0.934744698006713

Т.е. после всех событий произвели покупку 93.4% уникальных пользователей в текущей выборке, и почти 7% проигнорировало.  
В целом это очень хороший показатель.

Выводы:  
Столбец  date имеет неверный формат данных - необходимо будет преобразовать.  
Столбец category_ids - надо будет преобразовывать и скорее всего это будет разряженная матрица, т.к. небольшой объем данных  
ну и потом с такой матрицей умеет работать xgboost который и будем использовать в предсказаниях

#### app_target

In [147]:
app_target = check_data(app_target)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49849 entries, 0 to 49848
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   client_id  49849 non-null  int64
 1   target     49849 non-null  int64
dtypes: int64(2)
memory usage: 779.0 KB


None

,client_id,target
15755,1515915625491116504,0
2457,1515915625468259443,0
1752,1515915625468200447,0
24144,1515915625551635880,1
40795,1515915625762624766,0


Series([], dtype: int64)

None

,count,mean,std,min,25%,50%,75%,max
client_id,49849.0,1.515916e+18,1.487947e+08,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18,1.515916e+18
target,49849.0,1.927822e-02,1.375025e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00


Выводы:  
Ну тут все понятно, обсуждать нечего

#### event_type

In [148]:
event_type = check_data(event_type)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131072 entries, 0 to 131071
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   date                 131072 non-null  object
 1   bulk_campaign_id     131072 non-null  int64 
 2   count_click          131072 non-null  int64 
 3   count_complain       131072 non-null  int64 
 4   count_hard_bounce    131072 non-null  int64 
 5   count_open           131072 non-null  int64 
 6   count_purchase       131072 non-null  int64 
 7   count_send           131072 non-null  int64 
 8   count_soft_bounce    131072 non-null  int64 
 9   count_subscribe      131072 non-null  int64 
 10  count_unsubscribe    131072 non-null  int64 
 11  nunique_click        131072 non-null  int64 
 12  nunique_complain     131072 non-null  int64 
 13  nunique_hard_bounce  131072 non-null  int64 
 14  nunique_open         131072 non-null  int64 
 15  nunique_purchase     131072 non-nu

None

,date,bulk_campaign_id,count_click,count_complain,count_hard_bounce,count_open,count_purchase,count_send,count_soft_bounce,count_subscribe,...,nunique_open,nunique_purchase,nunique_send,nunique_soft_bounce,nunique_subscribe,nunique_unsubscribe,count_hbq_spam,nunique_hbq_spam,count_close,nunique_close
8876,2022-07-12,1625,0,0,0,3,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
23990,2022-09-26,5548,0,0,0,52,0,0,0,0,...,51,0,0,0,0,1,0,0,0,0
106035,2024-01-31,13427,0,0,0,12,0,0,0,0,...,12,0,0,0,0,1,0,0,0,0
130768,2024-05-13,15138,2549,0,1195,121771,7,166706,158,0,...,119837,7,166706,158,0,0,0,0,0,0
94650,2023-12-02,14105,0,0,0,42,0,0,0,0,...,42,0,0,0,0,0,0,0,0,0


Series([], dtype: int64)

None

,count,mean,std,min,25%,50%,75%,max
bulk_campaign_id,131072.0,8416.743378,4877.369306,548.0,4116.0,7477.0,13732.0,15150.0
count_click,131072.0,90.982971,1275.503564,0.0,0.0,0.0,2.0,128453.0
count_complain,131072.0,0.932655,30.198326,0.0,0.0,0.0,0.0,5160.0
count_hard_bounce,131072.0,78.473434,1961.317826,0.0,0.0,0.0,0.0,287404.0
count_open,131072.0,3771.090691,65160.668444,0.0,1.0,6.0,30.0,5076151.0
count_purchase,131072.0,0.577927,9.107040,0.0,0.0,0.0,0.0,1077.0
count_send,131072.0,11634.142319,175709.508290,0.0,0.0,0.0,0.0,11543513.0
count_soft_bounce,131072.0,27.807312,736.944714,0.0,0.0,0.0,0.0,76284.0
count_subscribe,131072.0,0.140518,2.072777,0.0,0.0,0.0,0.0,189.0
count_unsubscribe,131072.0,6.362679,79.172069,0.0,0.0,0.0,1.0,9089.0


Вывод:  
Здесь просто агрегированная статистика по событиям, не вижу смысла, что-то делать с этой таблицей в принципе

#### event_chanel

In [149]:
event_chanel = check_data(event_chanel)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131072 entries, 0 to 131071
Data columns (total 36 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   date                             131072 non-null  object
 1   bulk_campaign_id                 131072 non-null  int64 
 2   count_click_email                131072 non-null  int64 
 3   count_click_mobile_push          131072 non-null  int64 
 4   count_open_email                 131072 non-null  int64 
 5   count_open_mobile_push           131072 non-null  int64 
 6   count_purchase_email             131072 non-null  int64 
 7   count_purchase_mobile_push       131072 non-null  int64 
 8   count_soft_bounce_email          131072 non-null  int64 
 9   count_subscribe_email            131072 non-null  int64 
 10  count_unsubscribe_email          131072 non-null  int64 
 11  nunique_click_email              131072 non-null  int64 
 12  nunique_click_mo

None

,date,bulk_campaign_id,count_click_email,count_click_mobile_push,count_open_email,count_open_mobile_push,count_purchase_email,count_purchase_mobile_push,count_soft_bounce_email,count_subscribe_email,...,count_send_email,nunique_hard_bounce_email,nunique_hbq_spam_email,nunique_send_email,count_soft_bounce_mobile_push,nunique_soft_bounce_mobile_push,count_complain_email,nunique_complain_email,count_close_mobile_push,nunique_close_mobile_push
11070,2022-07-23,4895,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33650,2022-11-09,7535,25,0,290,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9667,2022-07-16,2229,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107231,2024-02-05,14475,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
122365,2024-04-10,14540,0,0,11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Series([], dtype: int64)

None

,count,mean,std,min,25%,50%,75%,max
bulk_campaign_id,131072.0,8416.743378,4877.369306,548.0,4116.0,7477.0,13732.0,15150.0
count_click_email,131072.0,41.582169,745.484035,0.0,0.0,0.0,1.0,59365.0
count_click_mobile_push,131072.0,49.400803,1036.952898,0.0,0.0,0.0,0.0,128453.0
count_open_email,131072.0,423.706032,9753.383722,0.0,1.0,5.0,23.0,2597015.0
count_open_mobile_push,131072.0,3347.384659,64448.590783,0.0,0.0,0.0,0.0,5076151.0
count_purchase_email,131072.0,0.357483,8.287483,0.0,0.0,0.0,0.0,1077.0
count_purchase_mobile_push,131072.0,0.220444,3.796500,0.0,0.0,0.0,0.0,431.0
count_soft_bounce_email,131072.0,24.474823,727.069387,0.0,0.0,0.0,0.0,76284.0
count_subscribe_email,131072.0,0.140518,2.072777,0.0,0.0,0.0,0.0,189.0
count_unsubscribe_email,131072.0,6.362679,79.172069,0.0,0.0,0.0,1.0,9089.0


Вывод:  
По сути своей, тоже самое, что и прошлая таблица - статистика по событиям.  
Возможно и есть смысл где-то использовать, но пока непонятно где и как

#### Выводы

Были подгружены и изучены предоставленные данные.  
Глобально, для реализации задачи нам понадобится только 3 таблицы - apparel-messages, apparel-purchases и apparel-target_binary, т.к. эти таблицы несут основную смысловую нагрузку.  
Две оставшиеся таблицы - статистика по ивентам и активности пользователей без привязки к этим самым пользователям и ничего нам не дадут.  



## Предобработка данных

### Обработка имеющихся данных

In [150]:
# преобразуем даты
app_msg["date"] = pd.to_datetime(app_msg["date"], errors="coerce")
app_msg["created_at"] = pd.to_datetime(app_msg["created_at"], errors="coerce")

app_prch["date"] = pd.to_datetime(app_prch["date"], errors="coerce")

### feature engineering

Для начала поработаем с каждой таблицей по отдельности и сделаем, что-то новое.  

app_msg
Что мы можем сделать:  
1) У нас есть дата реакции на ивент и дата создания ивента - скорость реакции на ивент, потом усредняем;  
2) Есть канал и действие - соберем суммарное число действий по каждому каналу.  

app_prch  
Что мы можем сделать:  
1) Время с момента последней покупки, сделаем в днях;  
2) Среднее число товаров в заказе;  
3) Средний чек заказа;  
4) В какой категории больше всего покупок (предварительно разобьем category_ids на top и last id - глобальную категорию и конкретный товар);  
5) Любимый товар;  
6) Средний интервал между покупками, в днях;  
7) Дней с последнего взаимодействия с рассылкой;  
8) Сделаем группировку по 30/60/90/180/360 дней, а там агрегируем по числу покупок, числу итемов, сумме затрат от последней имеющейся у нас отчетной даты;  

In [ ]:
app_msg.head()

In [ ]:
# в текущем виде данные не подходят для обучения, нужно агрегировать
app_msg_agg = (
    app_msg.groupby("client_id")
    .agg(
        bulk_campaigns=("bulk_campaign_id", "nunique"),             # сколько кампаний видел
        messages=("message_id", "nunique"),                         # сколько сообщений получил
        events=("event", "nunique"),                                # сколько уникальных событий
        channels=("channel", "nunique"),                            # через сколько каналов общались
        first_date=("date", "min"),                                 # первая дата активности
        last_date=("date", "max"),                                  # последняя дата активности
        pop_event=("event", lambda x: x.value_counts().idxmax())    # и самая популярная активность у бзера
    )
    .reset_index()
)

In [ ]:
display(app_msg_agg.head())

In [ ]:
# разберем category_ids
# превращаем в списки category_ids
app_prch["category_ids"] = app_prch["category_ids"].apply(parse_category_ids)

# берём первую категорию
app_prch["category_ids_top"] = app_prch["category_ids"].apply(
    lambda x: x[0] if len(x) > 0 else None
).astype("Int16")

# берём последнюю непустую категорию
app_prch["category_ids_last"] = app_prch["category_ids"].apply(
    lambda x: next((i for i in reversed(x) if i not in [None, "", "nan"]), None)
).astype("Int16")

In [ ]:
# собираем в кучу
app_prch_agg = (
    app_prch.groupby("client_id")
    .agg(
        quantity_mean=("quantity", "mean"),                                         # среднее число товаров в чеке
        price_mean=("price", "mean"),                                               # средняя цена позиции
        last_date=("date", "max"),                                                  # дата последней покупки
        last_item=("category_ids_last", "last"),                                    # последний купленный товар
        last_cat_glob=("category_ids_top", "last"),                                 # последняя купленная глобальная категория
        unique_cat_count=("category_ids", lambda x: len(set(i for sub in x for i in sub))),  # число уникальных категорий
    )
    .reset_index()
)

In [ ]:
display(app_prch_agg.info())
display(app_prch_agg.head())